# Backup Helper


In [5]:
PASS="GIT_CRYPT"
USER="GIT_CRYPT"
DIR="BackupVideo"

#feature flags for full backup (readonly) or incremental, DEBUG
DEBUG=False
FULL=False
BUILDKIT=True
# generate Dockerfile from one flat directory
GEN=False

In [7]:
import os
from hashlib import sha256
if DEBUG:
    print("USER", ":", USER, ", PASS", ":",
          sha256(PASS.encode('utf-8')).hexdigest())

In [17]:
if GEN:
    os.system('''
        cd {dir} && echo "FROM scratch" > Dockerfile &&
        for i in `ls -f *.*`;
          do 
            echo "ADD \'$i\' /backup" >> Dockerfile; 
          done  
        '''.format(dir=DIR))

## below can fail as default password is alway with invalid token

In [9]:
import sys
!{sys.executable} -m pip install docker

import docker, datetime
if BUILDKIT:
    os.environ.update({"DOCKER_BUILDKIT":"1"})
client = docker.from_env()

%time client.login(username=USER, password=PASS, reauth=True)

CPU times: user 2.72 ms, sys: 654 µs, total: 3.37 ms
Wall time: 9.9 s


{'IdentityToken': '', 'Status': 'Login Succeeded'}

In [10]:
timestamp=datetime.date.today().strftime("%Y%m%d%H%sb")
if FULL:
    TAG="kenney/%s-%s" % (DIR, timestamp)
else:
    TAG="kenney/mobile:%s-%s" % (DIR, timestamp)

VALIDTAG=TAG.lower()
try:
    client.images.build(tag=VALIDTAG, path=DIR)
except TypeError:
    print("path has no dockerfile")
    sys.exit(1)
except:
    print("repo must be lowercase/valid syntax")
    sys.exit(1)

In [6]:
try:
    output = client.images.push(repository=VALIDTAG)
except docker.errors.ReadTimeoutError:
    raise BaseException("timeout")
except:
    raise BaseException("unknown")
# TODO check syntax of juypter
# TODO clean up same interview name image
if DEBUG:
    print(output)

'{"status":"The push refers to repository [docker.io/kenney/interviews-20201224001608796800]"}\r\n{"status":"Preparing","progressDetail":{},"id":"e6891b5363de"}\r\n{"errorDetail":{"message":"denied: requested access to the resource is denied"},"error":"denied: requested access to the resource is denied"}\r\n'

In [ ]:
if "denied" in output:
    raise BaseException("denied")